# Demo Notebook For Training and Running Zingg Entity Resolution Workflow on Databricks
This notebook runs the Zingg Febrl Example on Databricks. 
Please refer to the 
1. [Zingg Python API](https://readthedocs.org/projects/zingg/) 
2. [Zingg Official Documentation](https://www.docs.zingg.ai) 
for details.

Please ensure to follow these prerequisites
1. Cluster installed with Zingg from pypi
2. Zingg jar from the [repo](https://github.com/zinggAI/zingg/releases)
, recommended(0.4.0)
 installed in the cluster library
3. tabulate from pypi
4. Make sure your databricks cluster is spinned up with runtime version 15.4 LTS
5. ~~Input csv file been added to DBFS (example csv can be found at ./examples/febrl/test.csv)~~
A volume in Unity Catalog for the zingg data. Example: `/Volumes/main/mike_echevarria/zingg`. Add `test.csv` to that volume

Please execute each cell one by one as per the instructions provided.

If you face any issue, please [log an issue](https://github.com/zinggAI/zingg/issues)

You can also join [Zingg's Slack community](https://join.slack.com/t/zinggai/shared_invite/zt-w7zlcnol-vEuqU9m~Q56kLLUVxRgpOA)

In [0]:
pip install zingg

In [0]:
dbutils.library.restartPython()

In [0]:
pip show zingg

In [0]:
pip install tabulate

In [0]:
dbutils.library.restartPython()

In [0]:
pip show tabulate

# Define locations for the model
The Zingg models and training data are persisted in dbfs. 

**Please edit the model id in the cell below to reflect your model.**

In [0]:
##you can change these to the locations of your choice
##these are the only two settings that need to change
#zinggDir = "/models"
zinggDir = "/Volumes/main/mike_echevarria/zingg"
modelId = "testModelFebrl"

# Setup common functions for use in Zingg. 
These functions setup the internal folders used by Zingg, and help with labeling and training Zingg. 

**No change is needed in the cell below.**

In [0]:

##please leave the following unchanged
MARKED_DIR = zinggDir + "/" + modelId + "/trainingData/marked/"
UNMARKED_DIR = zinggDir + "/" + modelId + "/trainingData/unmarked/"

#MARKED_DIR_DBFS = "/dbfs" + MARKED_DIR
#UNMARKED_DIR_DBFS = "/dbfs" + UNMARKED_DIR  


import pandas as pd
import numpy as np
 
import time
import uuid
 
from tabulate import tabulate
from ipywidgets import widgets, interact, GridspecLayout
import base64

import pyspark.sql.functions as fn

##this code sets up the Zingg Python interface
from zingg.client import *
from zingg.pipes import *

def cleanModel():
    dbutils.fs.rm(MARKED_DIR, recurse=True)
    # drop unmarked data
    dbutils.fs.rm(UNMARKED_DIR, recurse=True)
    return

# assign label to candidate pair
def assign_label(candidate_pairs_pd, z_cluster, label):
  '''
  The purpose of this function is to assign a label to a candidate pair
  identified by its z_cluster value.  Valid labels include:
     0 - not matched
     1 - matched
     2 - uncertain
  '''
  
  # assign label
  candidate_pairs_pd.loc[ candidate_pairs_pd['z_cluster']==z_cluster, 'z_isMatch'] = label
  
  return
 
def count_labeled_pairs(marked_pd):
  '''
  The purpose of this function is to count the labeled pairs in the marked folder.
  '''

  n_total = len(np.unique(marked_pd['z_cluster']))
  n_positive = len(np.unique(marked_pd[marked_pd['z_isMatch']==1]['z_cluster']))
  n_negative = len(np.unique(marked_pd[marked_pd['z_isMatch']==0]['z_cluster']))
  
  return n_positive, n_negative, n_total

# setup widget 
available_labels = {
    'No Match':0,
    'Match':1,
    'Uncertain':2
    }
#dbutils.widgets.dropdown('label', 'Uncertain', available_labels.keys(), 'Is this pair a match?')




# Start building the Zingg program 
The following cell sets up the initial arguments for Zingg. 

**No change is needed in the cell below.**

In [0]:

#build the arguments for zingg
args = Arguments()
# Set the modelid and the zingg dir. You can use this as is
args.setModelId(modelId)
args.setZinggDir(zinggDir)


# Define the input
Our data is in csv so we provide a schema. You can choose other formats like parquet by using Pipe with parquet as the format.
You can also pass in a dataframe by using a Pipe with the in memory format. 
Please refer to [Pipes](https://zingg.readthedocs.io/en/latest/zingg.html#zingg.pipes.Pipe) for details on different formats

**Please modify this for your data.**

In [0]:
input_file="/Volumes/main/mike_echevarria/zingg/test.csv"
schema = "rec_id string, fname string, lname string, stNo string, add1 string, add2 string, city string, areaCode string, state string, dob string, ssn string"
inputPipe = CsvPipe("zingg_input", input_file, schema)

args.setData(inputPipe)

# Configure the output
Here we configure the putput to be a csv, but similar to the input above, the output can be a file format like parquet or delta or a data store like MySQL

**Please modify this for your data.**

In [0]:
#setting outputpipe in 'args'
outputPipe = CsvPipe("resultFebrl", f"{zinggDir}/febrlOutputTest")
args.setOutput(outputPipe)

# Define the match fields and their types

The cell below is used to configure Zingg with the fields for use in matching and the match types.
Details on the field definitions can be found at [Zingg official docs](https://www.docs.zingg.ai)

**Please modify this for your data.**

In [0]:
#set field definitions 
#please change these 
rec_id = FieldDefinition("rec_id", "string", MatchType.DONT_USE) 
fname = FieldDefinition("fname", "string", MatchType.FUZZY)
lname = FieldDefinition("lname", "string", MatchType.FUZZY)
stNo = FieldDefinition("stNo", "string", MatchType.FUZZY)
add1 = FieldDefinition("add1", "string", MatchType.FUZZY)
add2 = FieldDefinition("add2", "string", MatchType.FUZZY)
city = FieldDefinition("city", "string", MatchType.FUZZY)
areaCode = FieldDefinition("areaCode", "string", MatchType.FUZZY)
state = FieldDefinition("state", "string", MatchType.FUZZY)
dob = FieldDefinition("dob", "string", MatchType.EXACT)
ssn = FieldDefinition("ssn", "string", MatchType.EXACT) 

fieldDefs = [rec_id, fname, lname, stNo, add1, add2, city, areaCode, state, dob, ssn]
args.setFieldDefinition(fieldDefs)

# Performance settings

The numPartitions define how data is split across the cluster. Please change this as per your data and cluster size by referring to the performance section of the Zingg docs.
The labelDataSampleSize is used for sampling in findTrainingData. It lets Zingg select pairs for labeling in a reasonable amount of time. 
If the findTrainingData phase is taking to much time, please reduce this by atleast 1/10th of its previous value and try again.

**Please modify this for your data.**

In [0]:

# The numPartitions define how data is split across the cluster. 
# Please change the fllowing as per your data and cluster size by referring to the docs.

args.setNumPartitions(4)
args.setLabelDataSampleSize(0.5)



# Get Zingg to find pairs for user labeling
**No change is needed in the cell below.**

In [0]:
options = ClientOptions([ClientOptions.PHASE,"findTrainingData"])

#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.initAndExecute()

# Prepare for user labeling
**No change is needed in the cell below.**

In [0]:
options = ClientOptions([ClientOptions.PHASE,"label"])

#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.init()

# See if we have records for labeling

**No change is needed to the cell below.**

In [0]:
# get candidate pairs
candidate_pairs_pd = getPandasDfFromDs(zingg.getUnmarkedRecords())
 
# if no candidate pairs, run job and wait
if candidate_pairs_pd.shape[0] == 0:
  print('No unlabeled candidate pairs found.  Run findTraining job ...')

else:
    # get list of pairs (as identified by z_cluster) to label 
    z_clusters = list(np.unique(candidate_pairs_pd['z_cluster'])) 

    # identify last reviewed cluster
    last_z_cluster = '' # none yet

    # print candidate pair stats
    print('{0} candidate pairs found for labeling'.format(len(z_clusters)))
  

# Start labeling to make Zingg learn how we want to match our data

**No change is needed to the cell below.**

In [0]:
# Label Training Set

# define variable to avoid duplicate saves
ready_for_save = False
print(candidate_pairs_pd)

# user-friendly labels and corresponding zingg numerical value
# (the order in the dictionary affects how displayed below)
LABELS = {
  'Uncertain':2,
  'Match':1,
  'No Match':0  
  }

# GET CANDIDATE PAIRS
# ========================================================
#candidate_pairs_pd = get_candidate_pairs()
n_pairs = int(candidate_pairs_pd.shape[0]/2)
# ========================================================

# DEFINE IPYWIDGET DISPLAY
# ========================================================
display_pd = candidate_pairs_pd.drop(
  labels=[
    'z_zid', 'z_prediction', 'z_score', 'z_isMatch', 'z_zsource'
    ], 
  axis=1)

# define header to be used with each displayed pair
html_prefix = "<p><span style='font-family:Courier New,Courier,monospace'>"
html_suffix = "</p></span>"
header = widgets.HTML(value=f"{html_prefix}<b>" + "<br />".join([str(i)+"&nbsp;&nbsp;" for i in display_pd.columns.to_list()]) + f"</b>{html_suffix}")

# initialize display
vContainers = []
vContainers.append(widgets.HTML(value=f'<h2>Indicate if each of the {n_pairs} record pairs is a match or not</h2></p>'))

# for each set of pairs
for n in range(n_pairs):

  # get candidate records
  candidate_left = display_pd.loc[2*n].to_list()
  print(candidate_left)
  candidate_right = display_pd.loc[(2*n)+1].to_list()
  print(candidate_right)

  # define grid to hold values
  html = ''

  for i in range(display_pd.shape[1]):

    # get column name
    column_name = display_pd.columns[i]

    # if field is image
    if column_name == 'image_path':

      # define row header
      html += '<tr>'
      html += '<td><b>image</b></td>'

      # read left image to encoded string
      l_endcode = ''
      if candidate_left[i] != '':
        with open(candidate_left[i], "rb") as l_file:
          l_encode = base64.b64encode( l_file.read() ).decode()

      # read right image to encoded string
      r_encode = ''
      if candidate_right[i] != '':
        with open(candidate_right[i], "rb") as r_file:
          r_encode = base64.b64encode( r_file.read() ).decode()      

      # present images
      html += f'<td><img src="data:image/png;base64,{l_encode}"></td>'
      html += f'<td><img src="data:image/png;base64,{r_encode}"></td>'
      html += '</tr>'

    elif column_name != 'image_path':  # display text values

      if column_name == 'z_cluster': z_cluster = candidate_left[i]

      html += '<tr>'
      html += f'<td style="width:10%"><b>{column_name}</b></td>'
      html += f'<td style="width:45%">{str(candidate_left[i])}</td>'
      html += f'<td style="width:45%">{str(candidate_right[i])}</td>'
      html += '</tr>'

  # insert data table
  table = widgets.HTML(value=f'<table data-title="{z_cluster}" style="width:100%;border-collapse:collapse" border="1">'+html+'</table>')
  z_cluster = None

  # assign label options to pair
  label = widgets.ToggleButtons(
    options=LABELS.keys(), 
    button_style='info'
    )

  # define blank line between displayed pair and next
  blankLine=widgets.HTML(value='<br>')

  # append pair, label and blank line to widget structure
  vContainers.append(widgets.VBox(children=[table, label, blankLine]))

# present widget
display(widgets.VBox(children=vContainers))
# ========================================================

# mark flag to allow save 
ready_for_save = True


# Save all the labels provided by the user 
No change is needed to the cell below.

In [0]:
if not ready_for_save:
  print('No labels have been assigned. Run the previous cell to create candidate pairs and assign labels to them before re-running this cell.')

else:

  # ASSIGN LABEL VALUE TO CANDIDATE PAIRS IN DATAFRAME
  # ========================================================
  # for each pair in displayed widget
  for pair in vContainers[1:]:

    # get pair and assigned label
    html_content = pair.children[1].get_interact_value() # the displayed pair as html
    user_assigned_label = pair.children[1].get_interact_value() # the assigned label

    # extract candidate pair id from html pair content
    start = pair.children[0].value.find('data-title="')
    if start > 0: 
      start += len('data-title="') 
      end = pair.children[0].value.find('"', start+2)
    pair_id = pair.children[0].value[start:end]



    # assign label to candidate pair entry in dataframe
    candidate_pairs_pd.loc[candidate_pairs_pd['z_cluster']==pair_id, 'z_isMatch'] = LABELS.get(user_assigned_label)
  # ========================================================

  # SAVE LABELED DATA TO ZINGG FOLDER
  # ========================================================
  # make target directory if needed
  dbutils.fs.mkdirs(MARKED_DIR)
  
  # save label assignments
  # save labels
  zingg.writeLabelledOutputFromPandas(candidate_pairs_pd,args)

  # count labels accumulated
  marked_pd_df = getPandasDfFromDs(zingg.getMarkedRecords())
  n_pos, n_neg, n_tot = count_labeled_pairs(marked_pd_df)
  print(f'You have accumulated {n_pos} pairs labeled as positive matches.')
  print(f'You have accumulated {n_neg} pairs labeled as not matches.')
  print("If you need more pairs to label, re-run the cell for 'findTrainingData'")
  # ========================================================  

  # save completed
  ready_for_save = False

# Persist the Zingg models by training on the labels and run them to predict matches
No change is needed to the cell below.

In [0]:
options = ClientOptions([ClientOptions.PHASE,"trainMatch"])

#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.initAndExecute()

# Voila! We are done. Lets check the results. 

**No change is needed to the cell below.**

In [0]:
outputDF = spark.read.csv(f"{zinggDir}/febrlOutputTest")

In [0]:

colNames = ["z_minScore", "z_maxScore", "z_cluster", "rec_id", "fname", "lname", "stNo", "add1", "add2", "city", "areaCode", "state", "dob", "ssn"]
outputDF.toDF(*colNames).show(100)

In [0]:
options = ClientOptions([ClientOptions.PHASE,"generateDocs"])

#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.initAndExecute()

In [0]:
DOCS_DIR = zinggDir + "/" + modelId + "/docs/"

In [0]:
# check that docs succesfully generated
dbutils.fs.ls('file:'+DOCS_DIR)

In [0]:
displayHTML(open(DOCS_DIR+"model.html", 'r').read())

In [0]:
displayHTML(open(DOCS_DIR+"data.html", 'r').read())

# Improve accuracy - ask Zingg to recommend stopwords for a particular column. 
These are common occuring words like pvt, ltd etc which occur across a lot of rows and do not add any meaning to the matching. You can read more about hese at [the Zingg docs](https://docs.zingg.ai/zingg-0.4.0/improving-accuracy/stopwordsremoval)

**In the following cell, we are asking for recommendations for the city column. Please edit as per your needs**

In [0]:
options = ClientOptions([ClientOptions.PHASE,"recommend", "--column", "city"])
args.setStopWordsCutoff(0.5)
#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.initAndExecute()

# View recommendations and configure them in the Zingg arguments.

Please check the FieldDefinition's setStopWords() method to set the stop words for each field

**We are using the recommendations as is, but you can massage the values, add or remove words that matter to the dataset to get the best results**

In [0]:
#dbutils.fs.ls(zinggDir+"/"+modelId+"/stopWords")
stopwordsForCity = spark.read.csv(zinggDir+"/"+modelId+"/stopWords/city")
stopwordsForCity.show()